In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#%%html
#<style>
#.output_wrapper, .output {
#    height:auto !important;
#    max-height:1000px;  /* your desired max-height here */
#}
#.output_scroll {
#    box-shadow:none !important;
#    webkit-box-shadow:none !important;
#}
#</style>

In [3]:
import sys
sys.path.append("..")

In [4]:
from optimus import Optimus
from optimus.profiler.functions import print_json as j

In [5]:
# Create optimus
op = Optimus(master="local[*]", app_name = "optimus1" )

In [6]:
df = op.read.json("100.json")
df = df.drop("_id")

In [7]:
from optimus.profiler.profiler import Profiler
profiler = Profiler()

In [9]:
col_name = "_id"
#df1 = df.cols().cast(("_id", "string"))
df.cols().count_na("_id")

['acceso_discapacitados', 'acceso_tercera_edad', 'address', 'advertiser', 'aire_acondicionado', 'alberca', 'amueblado', 'antiguedad_ano', 'antiguedad_anos', 'antiguedad_construccion', 'antiguedad_estrenar', 'area_de_juegos_infantiles', 'asador', 'bano', 'banos', 'bodega', 'bodega(s)', 'calentador_de_agua', 'casa', 'casa_uso_suelo', 'centros_comerciales_cercanos', 'chimenea', 'cisterna', 'city', 'cocina_integral', 'cuarto_de_tv', 'cuartos_de_servicio', 'currency', 'departamento', 'description', 'drenaje', 'elevador(es)', 'escuelas_cercanas', 'estacionamiento', 'estacionamiento_de_visitas', 'estacionamientos', 'estado_de_conservacion', 'estudio', 'forma_del_terreno', 'frente_a_parque', 'fumadores', 'gas', 'gimnasio', 'is_project', 'jacuzzi', 'linea_telefonica', 'local', 'm2', 'm2_de_bodega', 'm2_de_fondo', 'm2_de_frente', 'm2_de_jardin', 'm2_superficie_construida', 'm2_superficie_total', 'mantenimiento_mn', 'mascotas', 'medio_bano', 'medios_banos', 'nave', 'niveles_construidos', 'numero_

ValueError: 'missing_columns' must be 'acceso_discapacitados', 'acceso_tercera_edad', 'address', 'advertiser', 'aire_acondicionado', 'alberca', 'amueblado', 'antiguedad_ano', 'antiguedad_anos', 'antiguedad_construccion', 'antiguedad_estrenar', 'area_de_juegos_infantiles', 'asador', 'bano', 'banos', 'bodega', 'bodega(s)', 'calentador_de_agua', 'casa', 'casa_uso_suelo', 'centros_comerciales_cercanos', 'chimenea', 'cisterna', 'city', 'cocina_integral', 'cuarto_de_tv', 'cuartos_de_servicio', 'currency', 'departamento', 'description', 'drenaje', 'elevador(es)', 'escuelas_cercanas', 'estacionamiento', 'estacionamiento_de_visitas', 'estacionamientos', 'estado_de_conservacion', 'estudio', 'forma_del_terreno', 'frente_a_parque', 'fumadores', 'gas', 'gimnasio', 'is_project', 'jacuzzi', 'linea_telefonica', 'local', 'm2', 'm2_de_bodega', 'm2_de_fondo', 'm2_de_frente', 'm2_de_jardin', 'm2_superficie_construida', 'm2_superficie_total', 'mantenimiento_mn', 'mascotas', 'medio_bano', 'medios_banos', 'n

In [10]:
Profiler.count_data_types(df, "antiguedad_construccion")

[Row(antiguedad_construccion_type='bool', count=3), Row(antiguedad_construccion_type='null', count=97)]


{'columns': {'antiguedad_construccion': {'details': {'bool': 3,
    'date': 0,
    'float': 0,
    'int': 0,
    'missing': 0,
    'null': 97,
    'string': 0},
   'type': 'bool'}},
 'count_types': {'bool': 1,
  'categorical': 0,
  'date': 0,
  'null': 0,
  'numeric': 0}}

In [11]:
df.cols().show("antiguedad_construccion")

BooleanType
+-----------------------+
|antiguedad_construccion|
+-----------------------+
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|


In [62]:
int(true)

NameError: name 'true' is not defined

In [23]:
Profiler.columns(df, "antiguedad_construccion")


[Row(antiguedad_construccion_type='bool', count=3), Row(antiguedad_construccion_type='null', count=97)]
cast function took 30.945 ms
count_na function took 1609.960 ms


{'columns': {'antiguedad_construccion': {'column_type': 'bool',
   'dtypes_stats': {'bool': 3,
    'date': 0,
    'float': 0,
    'int': 0,
    'missing': 0,
    'null': 97,
    'string': 0},
   'frequency': [{'count': 97, 'percentage': 97.0, 'value': None},
    {'count': 3, 'percentage': 3.0, 'value': True}],
   'missing_count': 97,
   'p_missing': 97.0,
   'p_uniques': 2.0,
   'uniques_count': 2}},
 'count_types': {'bool': 1,
  'categorical': 0,
  'date': 0,
  'null': 0,
  'numeric': 0},
 'rows_count': 100,
 'size': '9.2MiB'}

In [13]:
Profiler.run(df, "*")

cast function took 15.989 ms
cast function took 16.053 ms
cast function took 18.949 ms
cast function took 20.944 ms
cast function took 11.968 ms
cast function took 10.970 ms
cast function took 12.964 ms
count_na function took 3819.263 ms
[Row(acceso_discapacitados_type='bool', count=2), Row(acceso_discapacitados_type='string', count=2), Row(acceso_discapacitados_type='null', count=96)]
[Row(acceso_tercera_edad_type='bool', count=1), Row(acceso_tercera_edad_type='string', count=1), Row(acceso_tercera_edad_type='null', count=98)]
[Row(address_type='string', count=59), Row(address_type='null', count=41)]
[Row(advertiser_type='string', count=100)]
[Row(aire_acondicionado_type='bool', count=7), Row(aire_acondicionado_type='null', count=93)]
[Row(alberca_type='int', count=1), Row(alberca_type='bool', count=7), Row(alberca_type='string', count=4), Row(alberca_type='null', count=88)]
[Row(amueblado_type='bool', count=2), Row(amueblado_type='null', count=98)]
[Row(antiguedad_ano_type='float', c

count_na function took 1143.636 ms
count_na function took 1143.230 ms
count_na function took 1169.577 ms
count_na function took 1157.111 ms
count_na function took 1179.651 ms
min function took 1140.045 ms
max function took 1116.740 ms
drop_na function took 26.892 ms
cast function took 17.951 ms
percentile function took 178.479 ms
std function took 1141.822 ms
kurt function took 1176.955 ms
mean function took 1130.784 ms
drop_na function took 17.915 ms
cast function took 13.962 ms
median function took 109.670 ms
drop_na function took 3.954 ms
cast function took 4.987 ms
median function took 95.709 ms
mad function took 254.267 ms
skewness function took 1121.176 ms
sum function took 1107.164 ms
variance function took 1115.452 ms
count_zeros function took 1153.418 ms
cast function took 14.919 ms
count_na function took 1149.948 ms
min function took 1100.679 ms
max function took 1105.422 ms
drop_na function took 16.929 ms
cast function took 13.952 ms
percentile function took 104.714 ms
std f

std function took 1110.621 ms
kurt function took 1126.300 ms
mean function took 1096.912 ms
drop_na function took 18.951 ms
cast function took 20.941 ms
median function took 107.679 ms
drop_na function took 4.023 ms
cast function took 5.984 ms
median function took 86.767 ms
mad function took 253.295 ms
skewness function took 1112.744 ms
sum function took 1104.658 ms
variance function took 1111.732 ms
count_zeros function took 1165.422 ms
cast function took 12.927 ms
count_na function took 1154.485 ms
min function took 1123.076 ms
max function took 1111.693 ms
drop_na function took 15.924 ms
cast function took 12.997 ms
percentile function took 95.742 ms
std function took 1103.475 ms
kurt function took 1122.751 ms
mean function took 1109.535 ms
drop_na function took 16.915 ms
cast function took 12.965 ms
median function took 96.703 ms
drop_na function took 3.021 ms
cast function took 6.952 ms
median function took 89.757 ms
mad function took 239.354 ms
skewness function took 1109.482 ms


median function took 99.841 ms
mad function took 278.378 ms
skewness function took 1111.343 ms
sum function took 1092.080 ms
variance function took 1128.686 ms
count_zeros function took 1147.588 ms
cast function took 14.961 ms
count_na function took 1160.410 ms
min function took 1173.076 ms
max function took 1131.726 ms
drop_na function took 22.939 ms
cast function took 14.983 ms
percentile function took 125.662 ms
std function took 1121.641 ms
kurt function took 1127.153 ms
mean function took 1125.669 ms
drop_na function took 20.887 ms
cast function took 14.959 ms
median function took 114.635 ms
drop_na function took 3.027 ms
cast function took 7.997 ms
median function took 115.489 ms
mad function took 288.026 ms
skewness function took 1110.037 ms
sum function took 1111.731 ms
variance function took 1090.869 ms
count_zeros function took 1170.183 ms
cast function took 11.968 ms
count_na function took 1147.696 ms
min function took 1176.448 ms
max function took 1159.201 ms
drop_na functi

Number of columns,89
Number of rows,100
Total Missing (%),66.51%
Total size in memory,6.8MiB
Categorical,32
Numeric,29
Date,0
Bool,28
Not available,0
Distinct count,3
Unique (%),3.0


In [40]:
Profiler.count_data_types(df, "escuelas_cercanas")

[Row(escuelas_cercanas_type='bool', count=21), Row(escuelas_cercanas_type='string', count=3), Row(escuelas_cercanas_type='null', count=76)]


{'columns': {'escuelas_cercanas': {'details': {'bool': 21,
    'date': 0,
    'float': 0,
    'int': 0,
    'missing': 0,
    'null': 76,
    'string': 3},
   'type': 'bool'}},
 'count_types': {'bool': 1,
  'categorical': 0,
  'date': 0,
  'null': 0,
  'numeric': 0}}

In [41]:
from optimus.functions import filter_row_by_data_type as fbdt
col_name = "_id"
df.withColumn(col_name, fbdt(col_name, get_type=True)).show()


AnalysisException: "cannot resolve '`_id`' given input columns: [estado_de_conservacion, m2_de_jardin, bodega(s), casa_uso_suelo, m2_de_fondo, title, quinta, preciorenta_mn, salon_de_fiestas, precioventa_mn, amueblado, asador, m2_superficie_total, mantenimiento_mn, city, zone, preciotraspaso, acceso_discapacitados, antiguedad_construccion, cuartos_de_servicio, antiguedad_estrenar, ubicacion, description, estacionamiento, medio_bano, servicios_basicos_(agua/luz), m2, recamaras, estudio, address, oficina, estacionamientos, antiguedad_ano, departamento, nave, bodega, url, referer, numero_de_frentes, mascotas, bano, centros_comerciales_cercanos, fumadores, frente_a_parque, alberca, forma_del_terreno, aire_acondicionado, antiguedad_anos, niveles_construidos, gimnasio, medios_banos, m2_de_bodega, price, uso_de_suelo, preciorenta, recamara, property_type, drenaje, is_project, cuarto_de_tv, m2_de_frente, offer, cisterna, terreno_lote, salon_de_usos_multiples, area_de_juegos_infantiles, local, 

In [21]:
df.dtypes

[('acceso_discapacitados', 'string'),
 ('acceso_tercera_edad', 'string'),
 ('address', 'string'),
 ('advertiser', 'string'),
 ('aire_acondicionado', 'string'),
 ('alberca', 'string'),
 ('amueblado', 'string'),
 ('antiguedad_ano', 'bigint'),
 ('antiguedad_anos', 'bigint'),
 ('antiguedad_construccion', 'boolean'),
 ('antiguedad_estrenar', 'boolean'),
 ('area_de_juegos_infantiles', 'string'),
 ('asador', 'string'),
 ('bano', 'string'),
 ('banos', 'string'),
 ('bodega', 'string'),
 ('bodega(s)', 'string'),
 ('calentador_de_agua', 'string'),
 ('casa', 'string'),
 ('casa_uso_suelo', 'boolean'),
 ('centros_comerciales_cercanos', 'string'),
 ('chimenea', 'string'),
 ('cisterna', 'string'),
 ('city', 'string'),
 ('cocina_integral', 'string'),
 ('cuarto_de_tv', 'string'),
 ('cuartos_de_servicio', 'string'),
 ('currency', 'string'),
 ('departamento', 'string'),
 ('description', 'string'),
 ('drenaje', 'string'),
 ('elevador(es)', 'string'),
 ('escuelas_cercanas', 'string'),
 ('estacionamiento', '

In [52]:
a = df.cols().min("*")
print(a)

min function took 2042.048 ms
{'min_acceso_discapacitados': 'si', 'min_acceso_tercera_edad': 'si', 'min_address': ', ciudad ures, ures', 'min_advertiser': 'alejandra  gomez', 'min_aire_acondicionado': 'true', 'min_alberca': '1', 'min_amueblado': 'true', 'min_antiguedad_ano': 1, 'min_antiguedad_anos': 2, 'min_antiguedad_construccion': True, 'min_antiguedad_estrenar': True, 'min_area_de_juegos_infantiles': 'si', 'min_asador': 'si', 'min_bano': '1', 'min_banos': '2', 'min_bodega': 'comercial', 'min_bodega(s)': '1', 'min_calentador_de_agua': 'si', 'min_casa': 'true', 'min_casa_uso_suelo': True, 'min_centros_comerciales_cercanos': 'si', 'min_chimenea': 'true', 'min_cisterna': 'si', 'min_city': 'bahia de banderas', 'min_cocina_integral': 'si', 'min_cuarto_de_tv': 'si', 'min_cuartos_de_servicio': 'si', 'min_currency': 'MXN', 'min_departamento': 'true', 'min_description': '**remate por parte del banco**\n             \n             \n            un remate bancario te permite adquirir una propi

In [56]:
from pyspark.sql import functions as F
r =df.cols()._agg([F.min, F.max, F.stddev, F.kurtosis, F.mean, F.skewness, F.sum, F.variance], "m2_superficie_construida")
#r =df.cols()._agg([F.min], "*")
#r =df.cols()._agg([F.max], "*")
print(r)

_agg function took 1356.768 ms
{'min_m2_superficie_construida': 20, 'max_m2_superficie_construida': 1800000, 'stddev_m2_superficie_construida': 215045.99326870567, 'kurtosis_m2_superficie_construida': 65.00270410371745, 'mean_m2_superficie_construida': 26575.942857142858, 'skewness_m2_superficie_construida': 8.185144541561508, 'sum_m2_superficie_construida': 1860316, 'variance_m2_superficie_construida': 46244779220.92421}


In [37]:
type(F.min)

function